# Quick Data Extraction (250 Games)

Extract 250 games from 2023-24 season with all covariates.
Uses ~500 API calls (under the 500-600/hour limit).

**Estimated time:** 6-7 minutes

In [ ]:
import pandas as pd
import numpy as np
import time
from datetime import datetime
from pathlib import Path
from nba_api.stats.endpoints.leaguegamefinder import LeagueGameFinder
from nba_api.stats.endpoints.playbyplayv3 import PlayByPlayV3
from nba_api.stats.endpoints.boxscoretraditionalv3 import BoxScoreTraditionalV3
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def extract_game_data(game_id):
    """Extract flagrant fouls, game outcome, and box score statistics."""
    try:
        # Get play-by-play for flagrant fouls
        pbp_response = PlayByPlayV3(game_id=game_id)
        pbp = pbp_response.play_by_play.get_data_frame()

        # Extract flagrants by team
        flagrants = pbp[pbp['subType'].isin(['Flagrant Type 1', 'Flagrant Type 2'])]
        home_flagrants = len(flagrants[flagrants['location'] == 'h'])
        away_flagrants = len(flagrants[flagrants['location'] == 'v'])

        # Get final score
        final_row = pbp.iloc[-1]
        home_score = final_row['scoreHome']
        away_score = final_row['scoreAway']

        # Get team IDs
        team_rows = pbp[pbp['teamId'] != 0]
        home_team = team_rows[team_rows['location'] == 'h']['teamId'].iloc[0]
        away_team = team_rows[team_rows['location'] == 'v']['teamId'].iloc[0]

        # Get box score statistics
        box_response = BoxScoreTraditionalV3(game_id=game_id)
        box_stats = box_response.box_score_player_stats.get_data_frame()

        # Aggregate team-level statistics
        team_stats = box_stats.groupby('teamId').agg({
            'reb': 'sum',
            'ast': 'sum',
            'tov': 'sum',
            'ftm': 'sum',
            'fta': 'sum'
        }).to_dict('index')

        home_stats = team_stats.get(home_team, {})
        away_stats = team_stats.get(away_team, {})

        # Count inactive players
        home_inactive = len(box_stats[(box_stats['teamId'] == home_team) & (box_stats['min'] == 0)])
        away_inactive = len(box_stats[(box_stats['teamId'] == away_team) & (box_stats['min'] == 0)])

        return {
            'game_id': str(game_id),
            'home_team': int(home_team),
            'away_team': int(away_team),
            'home_flagrants': int(home_flagrants),
            'away_flagrants': int(away_flagrants),
            'home_score': int(home_score),
            'away_score': int(away_score),
            'home_rebounds': int(home_stats.get('reb', 0)),
            'away_rebounds': int(away_stats.get('reb', 0)),
            'home_assists': int(home_stats.get('ast', 0)),
            'away_assists': int(away_stats.get('ast', 0)),
            'home_turnovers': int(home_stats.get('tov', 0)),
            'away_turnovers': int(away_stats.get('tov', 0)),
            'home_ftm': int(home_stats.get('ftm', 0)),
            'away_ftm': int(away_stats.get('ftm', 0)),
            'home_fta': int(home_stats.get('fta', 0)),
            'away_fta': int(away_stats.get('fta', 0)),
            'home_inactive_players': int(home_inactive),
            'away_inactive_players': int(away_inactive)
        }, None
    except Exception as e:
        return None, e

In [ ]:
# Get 2023-24 season games
print("Fetching game IDs for 2023-24 season...")
gamefinder = LeagueGameFinder(season_nullable='2023-24')
games_df = gamefinder.get_data_frames()[0]
all_game_ids = games_df['GAME_ID'].unique().tolist()

# Limit to 250 games (500 API calls)
MAX_GAMES = 250
game_ids = all_game_ids[:MAX_GAMES]

print(f"Total games in 2023-24: {len(all_game_ids)}")
print(f"Extracting: {len(game_ids)} games")
print(f"API calls needed: ~{len(game_ids)*2}")
print(f"Estimated time: {(len(game_ids) * 1.5 / 60):.1f} minutes\n")

In [ ]:
# Extract data
print(f"Starting extraction at {datetime.now().strftime('%H:%M:%S')}\n")

successful_count = 0
failed_count = 0
all_games = []

for i, game_id in enumerate(game_ids):
    if (i + 1) % 25 == 0:
        print(f"Progress: {i+1}/{len(game_ids)} | Success: {successful_count} | Errors: {failed_count} | Time: {datetime.now().strftime('%H:%M:%S')}")

    game_data, error = extract_game_data(game_id)

    if game_data:
        all_games.append(game_data)
        successful_count += 1
    else:
        print(f"  Error on game {game_id}: {error}")
        failed_count += 1

    # 1.5 second throttle
    time.sleep(1.5)

print(f"\nExtraction completed at {datetime.now().strftime('%H:%M:%S')}")
print(f"Successfully extracted: {successful_count} games")
print(f"Errors: {failed_count} games")

In [ ]:
# Save to CSV
if all_games:
    df = pd.DataFrame(all_games)
    csv_file = Path('nba_flagrant_fouls.csv')
    df.to_csv(csv_file, index=False)
    
    print(f"\nData saved to: {csv_file}")
    print(f"Total rows: {len(df)}")
    print(f"\nColumns ({len(df.columns)}):")
    print(df.columns.tolist())
    
    print(f"\nFirst few rows:")
    display(df.head())
    
    print(f"\nDescriptive statistics:")
    display(df.describe())
else:
    print("No data extracted!")